In [18]:
import pandas as pd

In [34]:
taxi = pd.read_csv("taxi_nyc_lesson_2.csv")

I have checked the number of total rows and columns in the dataset.

In [36]:
taxi.shape

(29101, 14)

I have looked at the types of columns.

In [38]:
taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29101 entries, 0 to 29100
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pickup_dt     29101 non-null  object 
 1   pickup_month  29101 non-null  object 
 2   borough       26058 non-null  object 
 3   pickups       29101 non-null  int64  
 4   hday          29101 non-null  object 
 5   spd           29101 non-null  float64
 6   vsb           29101 non-null  float64
 7   temp          29101 non-null  float64
 8   dewp          29101 non-null  float64
 9   slp           29101 non-null  float64
 10  pcp 01        29101 non-null  float64
 11  pcp 06        29101 non-null  float64
 12  pcp 24        29101 non-null  float64
 13  sd            29101 non-null  float64
dtypes: float64(9), int64(1), object(4)
memory usage: 3.1+ MB


The column names pcp 01, pcp 06, and pcp 24 contain spaces, which is not very convenient for further access to the columns using dot notation without the use of quotes and parentheses. So I replaced the space in the names with an underscore.

In [21]:
taxi = taxi.rename(columns={'pcp 01': 'pcp_1', 'pcp 06': 'pcp_06', 'pcp 24': 'pcp_24'})

In [39]:
taxi.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


It is necessary to find out how many records (rows) in the data frame relate to different city districts.

In [40]:
taxi['borough'].value_counts()

Staten Island    4343
Brooklyn         4343
Bronx            4343
EWR              4343
Manhattan        4343
Queens           4343
Name: borough, dtype: int64

The next task is to find out which district had the largest number of trips during the entire period. To start with, calculate the total number of trips (pickups) without grouping.

In [24]:
taxi.pickups.sum()

14265773

In [47]:
taxi_city = taxi.groupby('borough') \
                .agg({'pickups': 'sum'})
taxi_city

,pickups
borough,
Bronx,220047
Brooklyn,2321035
EWR,105
Manhattan,10367841
Queens,1343528
Staten Island,6957


In [48]:
taxi_city.idxmin()

pickups    EWR
dtype: object

In [49]:
taxi_city.idxmax()

pickups    Manhattan
dtype: object

To continue the data exploration, let's look at the number of rides on weekends. I grouped the data by two features: city district and whether the day is a holiday (borough and hday columns). I compared the average number of rides and selected the districts that receive more orders on holidays, on average, than on regular days.

In [50]:
taxi_day = taxi.groupby(['borough', 'hday']) \
                .agg({'pickups': 'mean'})
taxi_day

pickups
borough       hday             
Bronx         N       50.771073
              Y       48.065868
Brooklyn      N      534.727969
              Y      527.011976
EWR           N        0.023467
              Y        0.041916
Manhattan     N     2401.302921
              Y     2035.928144
Queens        N      308.899904
              Y      320.730539
Staten Island N        1.606082
              Y        1.497006

For each district, I counted the number of trips per month. Then I sorted the obtained values in descending order.

In [51]:
pickups_by_mon_bor = taxi.groupby(['pickup_month', 'borough']) \
                .agg({'pickups': sum}) \
                .sort_values('pickups', ascending=False)
pickups_by_mon_bor

,,pickups
pickup_month,borough,
Jun,Manhattan,1995388
May,Manhattan,1888800
Feb,Manhattan,1718571
Mar,Manhattan,1661261
Apr,Manhattan,1648278
Jan,Manhattan,1455543
Jun,Brooklyn,482466
May,Brooklyn,476087
Apr,Brooklyn,378095


The temperature in the data about taxi rides in New York is presented in Fahrenheit degrees. A function named temp_to_celcius was written, which takes as input a column of temperature in °F and returns the values converted to Celsius degrees.

In [32]:
def temp_to_celcius (temp):
    temp_f = (temp - 32) * 5.0 / 9.0
    return temp_f